In [1]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-5e6ksres
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-5e6ksres
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 12.2MB/s 
     |████████████████████████████████| 901kB 49.2MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=a12df36900a63d961d26f446d1dd466818cdf0b3ae93c65519c087b954ac1e81
  Stored in directory: /tmp/pip-ephem-wheel-cache-vxf4so4t/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 12.9MB/s 
     |████████████████████████████████| 245kB 20.3MB/s 
     |████████████████████████████████| 112kB 26.9MB/s 


In [2]:
import torch
import numpy
import matplotlib.pyplot as plt
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import argparse
import pdb

from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
Tesla V100-SXM2-16GB


In [3]:
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/SCIREC/output/step_three/mlm/",
    "adapter_name": "SCIREC_adapter"
}

In [4]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [6]:
train_data = datasets.load_dataset("json", data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset("json", data_files=args_dict["val_dataset"])["train"] 

Using custom data configuration default-9f081ed18e6af576


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9f081ed18e6af576/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-be8b4aa859efaf0f


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-be8b4aa859efaf0f/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [7]:
def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True, return_special_tokens_mask=True)
    return tokenized_batch

train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data), remove_columns=["text", "label"])
train_data.set_format('torch', columns=['input_ids'])


val_data = val_data.map(tokenization, batched=True, batch_size=len(train_data), remove_columns=["text", "label"])
val_data .set_format('torch', columns=['input_ids'])

In [8]:
#static masking. Possible TODO: use dynamic masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
adapter_name = "mlm_SCIREC_pretrain"
model.add_adapter(adapter_name)
model.train_adapter(adapter_name)

In [12]:
training_args = TrainingArguments(
    num_train_epochs=120,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=1e-4,
    #lr_scheduler_type="constant",        
    adam_epsilon=1e-6,
    adam_beta1=0.9,
    adam_beta2=0.98,
    weight_decay=0.01,
    warmup_ratio=0.06,
    fp16=True,
    eval_accumulation_steps=1,
    save_steps=5000,
    save_total_limit=2,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    seed=3,
    output_dir=args_dict["output_dir"],
    evaluation_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data
)

In [13]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,5.555417,2.613000,174.129000
2,No log,5.657770,2.539500,179.167000
3,No log,6.099019,2.562400,177.566000
4,No log,5.787459,2.579900,176.365000
5,No log,5.774350,2.573300,176.813000
6,No log,5.677395,2.554300,178.131000
7,No log,5.649177,2.550100,178.425000
8,No log,5.512362,2.524200,180.255000
9,No log,5.530841,2.526100,180.120000
10,5.591500,5.436038,2.543000,178.925000


TrainOutput(global_step=6120, training_loss=3.566620730730443, metrics={'train_runtime': 3135.7868, 'train_samples_per_second': 1.952, 'total_flos': 3.638524702130988e+16, 'epoch': 120.0, 'init_mem_cpu_alloc_delta': 4096, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1482752, 'train_mem_gpu_alloc_delta': 12363264, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 9460497920})

In [14]:
 model.save_adapter(args_dict["output_dir"] + adapter_name, adapter_name)